[Magic](https://www.kaggle.com/jturkewitz/magic-features-0-03-gain)

In [1]:
import numpy as np
import pandas as pd
import timeit

In [2]:
train_orig =  pd.read_csv('../quora_pairs/data/train.csv', header=0)
test_orig =  pd.read_csv('../quora_pairs/data/test.csv', header=0)

In [10]:
test_orig.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [6]:
train_orig.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
df1 = train_orig[['question1']].copy()
df2 = train_orig[['question2']].copy()
df1_test = test_orig[['question1']].copy()
df2_test = test_orig[['question2']].copy()

df2.rename(columns = {'question2':'question1'},inplace=True)
df2_test.rename(columns = {'question2':'question1'},inplace=True)

In [7]:
train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)

In [9]:
len(train_questions)

5500172

In [11]:
#train_questions.drop_duplicates(subset = ['qid1'],inplace=True)
train_questions.drop_duplicates(subset = ['question1'],inplace=True)

train_questions.reset_index(inplace=True,drop=True)
questions_dict = pd.Series(train_questions.index.values,index=train_questions.question1.values).to_dict()

train_cp = train_orig.copy()
test_cp = test_orig.copy()
train_cp.drop(['qid1','qid2'],axis=1,inplace=True)

test_cp['is_duplicate'] = -1
test_cp.rename(columns={'test_id':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])

In [14]:
comb['q1_hash'] = comb['question1'].map(questions_dict)
comb['q2_hash'] = comb['question2'].map(questions_dict)

q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()

In [15]:
def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0
#map to frequency space
comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))

train_comb = comb[comb['is_duplicate'] >= 0][['id','q1_hash','q2_hash','q1_freq','q2_freq','is_duplicate']]
test_comb = comb[comb['is_duplicate'] < 0][['id','q1_hash','q2_hash','q1_freq','q2_freq']]

In [20]:
train_comb.head()

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
0,0,0,290457,1,2,0
1,1,1,16739,8,3,0
2,2,2,290458,2,1,0
3,3,3,290459,1,1,0
4,4,4,290460,3,1,0


In [21]:
print len(train_comb)
print len(test_comb)

404290
2345796


In [22]:
train_comb.to_csv("../quora_pairs/data/new_feature/train_magic_feat.csv",index=False)
test_comb.to_csv("../quora_pairs/data/new_feature/test_magic_feat.csv",index=False)

In [16]:
corr_mat = train_comb.corr()
corr_mat.head()

,id,q1_hash,q2_hash,q1_freq,q2_freq,is_duplicate
id,1.000000,0.692730,0.286969,-0.001608,-0.000777,-0.008784
q1_hash,0.692730,1.000000,0.492993,-0.341777,-0.202545,-0.206498
q2_hash,0.286969,0.492993,1.000000,-0.392605,-0.466434,-0.349626
q1_freq,-0.001608,-0.341777,-0.392605,1.000000,0.494315,0.296621
q2_freq,-0.000777,-0.202545,-0.466434,0.494315,1.000000,0.198609
